In [33]:
import os
%pwd
os.chdir("/home/om/Desktop/dl-project")
%pwd

'/home/om/Desktop/dl-project'

In [34]:
import pandas as pd
df = pd.read_csv("/home/om/Desktop/dl-project/artifacts/data_ingestion/kidneyData.csv")


In [35]:
df.columns

Index(['Unnamed: 0', 'image_id', 'path', 'diag', 'target', 'Class'], dtype='object')

In [36]:
df['Class'].unique()

array(['Tumor', 'Normal', 'Stone', 'Cyst'], dtype=object)

In [37]:
from dataclasses import dataclass

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: str
    base_model_path: str
    updated_model_path: str
    params_image_size: list
    params_weights: str
    params_learning_rate: float
    params_include_top: bool
    params_classes: int

In [38]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [39]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params.prepare_base_model

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=config.root_dir,
            base_model_path=config.base_model_path,
            updated_model_path=config.updated_model_path,
            params_image_size=params.IMAGE_SIZE,
            params_weights=params.WEIGHTS,
            params_learning_rate=params.LEARNING_RATE,
            params_include_top=params.INCLUDE_TOP,
            params_classes=params.CLASSES
        )
        return prepare_base_model_config

In [40]:
import os
from zipfile import ZipFile
import urllib.request as request
import tensorflow as tf
from cnnClassifier import logger

In [ ]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        logger.info("Preparing the base model...")
        # Download weights if needed, else use 'imagenet' or path
        weights = self.config.params_weights
        if weights == "imagenet":
            weights_arg = "imagenet"
        else:
            weights_arg = self.config.base_model_path

        base_model = tf.keras.applications.vgg16.VGG16(
            input_shape=tuple(self.config.params_image_size),
            include_top=self.config.params_include_top,
            weights=weights_arg
        )
        os.makedirs(os.path.dirname(self.config.base_model_path), exist_ok=True)
        base_model.save(self.config.base_model_path)
        
        return base_model

    def _prepare_full_model(self, classes, freeze_all, freeze_till, learning_rate):
        logger.info("Preparing the full model...")
        base_model = self.get_base_model()

        if freeze_all:
            for layer in base_model.layers:
                layer.trainable = False
        elif freeze_till is not None:
            for layer in base_model.layers[:freeze_till]:
                layer.trainable = False

        x = base_model.output
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(1024, activation='relu')(x)
        output = tf.keras.layers.Dense(classes, activation='softmax')(x)

        model = tf.keras.models.Model(inputs=base_model.input, outputs=output)

        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        model.summary()
        return model

    def update_base_model(self):
        logger.info("Updating the base model...")
        model = self._prepare_full_model(
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        os.makedirs(os.path.dirname(self.config.updated_model_path), exist_ok=True)
        model.save(self.config.updated_model_path)
        logger.info(f"Updated model saved at {self.config.updated_model_path}")


In [42]:
# pipeline code
try:
    config_manager = ConfigurationManager()
    prepare_base_model_config = config_manager.get_prepare_base_model_config()
    
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    logger.exception(e)
    raise e



2025-06-16 16:23:03,189 - INFO - common - Created directory: artifacts
2025-06-16 16:23:03,191 - INFO - 2097615327 - Preparing the base model...
58889256/58889256 [==============================] - 18s 0us/step
2025-06-16 16:23:22,481 - WARNING - saving_utils - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2025-06-16 16:23:22,550 - INFO - 2097615327 - Base model saved at artifacts/prepare_base_model/base_model.h5
2025-06-16 16:23:22,550 - INFO - 2097615327 - Updating the base model...
2025-06-16 16:23:22,551 - INFO - 2097615327 - Preparing the full model...
2025-06-16 16:23:22,551 - INFO - 2097615327 - Preparing the base model...
2025-06-16 16:23:22,807 - WARNING - saving_utils - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2025-06-16 16:23:22,927 - INFO - 2097615327 - Base mode